# An Introduction to Classification

* A Statistical Model Revisited

* Contingency Tables Revisited
    * odds
    * other hypothesis tests

* Logistic Regression
    * log odds
    * connection to linear regression
    * interpretation
    * parameters in scikit-learn
    * decision function
    * precision 
    * recall
    * f1 score
    * MCC
    * 

## A Statistical Model Revisited

Thus far we have looked at statistical models that carry out the regression task.  That is, they take in a set of one or more variables and produce a number.  Specifically, when we say regression we mean:

$$ \hat{y} = mX + b $$

On the right hand side:

Where `X` is a tensor of one or more variables.  When `X` represents a single variable, we call it a vector.  And when `X` represents more than one variable we typically refer to it as a matrix.  However, it is also possible for `X` to represent higher dimensions.

`m` and `b` are just scalars, typically from the real numbers.

On the left hand side:

$\hat{y}$ is also typically from the real numbers.  

And we say that we regress X on y.

One of the important things to note about this procedure is the nature of $\hat{y}$, because it is from the reals it's output carries distance.  That means:

if for a given set of X's $\hat{y}$ = 5.32 and for another set of X's $\hat{y}$ = -1.83 then we can say that the output of the first set of variables is strictly higher than the output of the second set.

It is not always the case that our output being metrizable, that is being measurable in terms of distance, is useful.  It may be the case that our output should not carry any sense of distance or comparison in anyway.

For this we need to introduce a new statistical task, that of classification.

## Classification

The basic idea behind classification is, what if we output a $\hat{y}$ that was categorical rather than continuous?  We've already seen categorical variables in the Applying Statistical Tests chapter.  But more formally, a categorical variable is one in which the different classes are just that, classes.  They are just designations.  So let's say we had two classes, A and B.  They could be classes of anything.  Like tall and short people.  Or young and old people.  Or different flavors of ice cream.  As much as people might try to rank order these different classes, neither is truly better than the other.  

If you want to try a fun experiment, ask some friends what they think about different classes of things, like maybe whether it's better to be young or old, better to be tall or short, better to eat vanilla or chocolate ice cream.  I bet, as long as your friends aren't too similar, they'll all answer differently.  And that's the point!  There is no objective ordering of any of these classes.  And therefore, we cannot define an explicit metric to rank them.

So what?  How are categorical variables useful?  Well turns out they have tons of uses!  We used them extensively in Applying Statistical Tests!  Specifically some of the demographic variables and the converted variable were all categorical.  Without categorical data, we'd never be able to model any of that!  And then we'd be greatly constraining the set of problems we can solve with statistical modeling and analysis.

Hopefully I've convinced you that classification is cool!  Now let's look at a basic definition of it, so we can compare against our regression task.

